In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
# contracts = populate_contracts(tickers)

In [3]:
from stockMarket.sec_edgar.downloader import Downloader

tickers = [ticker.replace(".", "-") for ticker in tickers]

dl = Downloader(tickers=tickers)
dl.download_10k_reports(fast_download=True)
dl.download_financial_statements_xlsx(from_year=2014)

  0%|          | 0/503 [00:00<?, ?it/s]/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:71: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000896878/000089687813000149/Financial_Report.xlsx for INTU
  warnings.warn(message, UserWarning)
/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:71: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000896878/000089687812000146/Financial_Report.xlsx for INTU
  warnings.warn(message, UserWarning)
/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:71: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000896878/000095013406017817/Financial_Report.xlsx for INTU
  warnings.warn(message, UserWarning)
/home/jag/Trading/stockMarket/stockMarket/sec_edgar/downloader.py:71: UserWarning: Failed to download https://www.sec.gov/Archives/edgar/data/0000896878/000089687811000093/Financial_Report.xlsx for INTU
  warnings.warn(message, UserWarnin

KeyboardInterrupt: 

In [177]:
import glob

from stockMarket.sec_edgar.financialReportReader import FinancialReportReader
from tqdm import tqdm
from zipfile import BadZipFile

key_combinations = []

for ticker in tqdm(sorted(tickers)):
# for ticker in tqdm(sorted(tickers)[492:]):
    files = glob.glob(f"../data/sec-edgar-filings/{ticker}/10-K/*/Financial_Report.xlsx")

    for file in files:
        if 14 < int(file.split("-")[-2]) <= 24:
            try:
                reader = FinancialReportReader(file)
                reader.read()
                key_combinations += reader.key_combinations
            except BadZipFile as e:
                print(f"BadZipFile: {ticker}")
                raise e

  0%|          | 0/502 [00:00<?, ?it/s]

100%|██████████| 502/502 [00:05<00:00, 83.85it/s]


In [137]:
from zipfile import BadZipFile

from stockMarket.sec_edgar.financialReportCollecter import FinancialReportCollecter

tickers = [ticker.replace(".", "-") for ticker in tickers if ticker != "KVUE"]

new_tickers = []
new_tickers.clear()
for ticker in sorted(tickers):
    if ticker in ["ZION", "WFC", "USB", "TFC", "SYF", "RF", "NTRS", "L", "KEY", "HBAN", "FITB", "DFS", "COF", "CMA", "MTB", "NVR", "AXP", "BAC", "BK", "C", "CARR", "CDW", "CFG"]:
        continue
    
    if ticker in ["FI", "FCX", "DOC", "ABNB", "HPQ", "INVH", "O", "GEHC", "LIN", "ALLE", "HLT", "KEYS", "BALL", "CF"]: # ABNB 2021 no data available and only 24, 23, 22 are available
        continue
    
    new_tickers.append(ticker)
    
collecter = FinancialReportCollecter(tickers=["GRMN"])
collecter.collect(update=True)
collecter.error_messages

100%|██████████| 1/1 [00:00<00:00, 1268.31it/s]


[]

In [139]:
import numpy as np

from stockMarket.sec_edgar.financialReportChecker import FinancialReportChecker
from tqdm import tqdm

correct_tickers = []

#correct until -18-
checked_tickers = []
checked_tickers += ["AAL"]
checked_tickers += ["ACGL"]
checked_tickers += ["ADI"]
checked_tickers += ["AFL"]
checked_tickers += ["AIG"]
checked_tickers += ["AIZ"]
checked_tickers += ["AJG"]
checked_tickers += ["ALL"]
checked_tickers += ["AMP"]
checked_tickers += ["APA"]
checked_tickers += ["ARE"]
checked_tickers += ["AVY"]

alphabetical_tickers = []
a_tickers = len(checked_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"A tickers: {a_tickers}")

checked_tickers += ["BA"]
checked_tickers += ["BAX"]
checked_tickers += ["BBWI"]
checked_tickers += ["BBY"]
checked_tickers += ["BF-B"]
checked_tickers += ["BLDR"]
checked_tickers += ["BRK-B"]
checked_tickers += ["BWA"]
checked_tickers += ["BX"]

b_tickers = len(checked_tickers) - len(alphabetical_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"B tickers: {b_tickers}")

checked_tickers += ["CB"]
checked_tickers += ["CBOE"]
checked_tickers += ["CBRE"]
checked_tickers += ["CCI"]
checked_tickers += ["CDNS"]
checked_tickers += ["CHTR"]
checked_tickers += ["CMCSA"]
checked_tickers += ["CME"]
checked_tickers += ["CNC"]
checked_tickers += ["CNP"]
checked_tickers += ["COP"]
checked_tickers += ["CPB"]
checked_tickers += ["CRM"]
checked_tickers += ["CVS"]
checked_tickers += ["CZR"]

c_tickers = len(checked_tickers) - len(alphabetical_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"C tickers: {c_tickers}")

checked_tickers += ["DE"]
checked_tickers += ["DG"]
checked_tickers += ["DIS"]
checked_tickers += ["DLTR"]
checked_tickers += ["DPZ"]
checked_tickers += ["DVN"]

d_tickers = len(checked_tickers) - len(alphabetical_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"D tickers: {d_tickers}")

checked_tickers += ["EBAY"]
checked_tickers += ["EG"]
checked_tickers += ["EL"]
checked_tickers += ["EQT"]

e_tickers = len(checked_tickers) - len(alphabetical_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"E tickers: {e_tickers}")

checked_tickers += ["FANG"]

f_tickers = len(checked_tickers) - len(alphabetical_tickers)
alphabetical_tickers += checked_tickers
alphabetical_tickers = list(set(alphabetical_tickers))
print(f"F tickers: {f_tickers}")

checked_tickers += ["GEN"]
checked_tickers += ["GILD"]
checked_tickers += ["GPN"]
checked_tickers += ["GS"]

deviation_messages = []

for ticker in tqdm(sorted(new_tickers)[193:]):
    try:
        checker = FinancialReportChecker(ticker)
        checked = checker.check_data()
    except Exception as e:
        if ticker in checked_tickers:
            continue
        print(f"Exception: {ticker}")
        raise e
    
    
    if not checked and ticker not in checked_tickers:
        for date,value in checker.wrong_data.items():
            print(f"{ticker} - {date}: {value[0], value[1]}")
        print(f"{ticker} - {checker.inconsistent_revenue}")
        break
    elif not checked:
        for date,value in checker.wrong_data.items():
            deviation_messages.append(f"{ticker} - {date}: {value[0], value[1]}")
    else:
        correct_tickers.append(ticker)
    

A tickers: 12
B tickers: 9
C tickers: 15
D tickers: 6
E tickers: 4
F tickers: 1


  1%|▏         | 4/272 [00:01<01:26,  3.09it/s]

HAS - 2023-12-31 00:00:00: (5003.3, 7583.3)
HAS - 2022-12-25 00:00:00: (5856.7, 6588.9)
HAS - 2020-12-27 00:00:00: (5465.443, 5929.5)
HAS - 2019-12-29 00:00:00: (4720.227, 6070.5)
HAS - 2018-12-30 00:00:00: (4579.646, 5753.3)
HAS - None


In [244]:
for ticker in correct_tickers:
    print(ticker)

A
AAL
AAPL
ABBV
ABT
